In [11]:
from bs4 import BeautifulSoup as bs
import requests 
import pandas as pd
import sqlite3
from flask import Flask, jsonify
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


wttj_url = 'https://www.welcometothejungle.com/fr/companies/hunteed/jobs/community-success-manager-community-builder_paris?q=6d2ebe57905edc93351a1b0e81bdf060&o=802986be-3cbd-4743-8bd2-b6830b8a75a2'
page = requests.get(wttj_url)
page.text 

soup = bs(page.text, "html.parser")
#if page.status_code == 200:
    #print(f"reussi de la requête. Code de statut: {page.status_code}")
#else:
   #@ print(f"Échec de la requête. Code de statut: {page.status_code}")
#soup

### Fonction qui récupère les informations d'une offre

In [12]:
def get_info(link):

    page = requests.get(link)
    soup = bs(page.text, "html.parser")
    company = soup.find('span', attrs = {'class' : "sc-empnci hmOCpj wui-text"}).string
    job = soup.find('h2', attrs = {'class': "sc-empnci cYPTxs wui-text"}).string
    lieu = soup.find('span', attrs = {'class': "sc-1eoldvz-2 kGQqJy"}).string
    spans = soup.find_all('span', attrs = {'class': "sc-bXCLTC kJcLKT"})
    try:
        if spans:
            start = spans[1].find_next("span").string
    except:
        start = "Date de prise de poste non trouvée"
    try : 
        if soup.find("time").has_attr('datetime'):
            debut= soup.find("time").string
    except : 
            debut = "Non trouvé"
        
    col = soup.find_all('span', attrs = {'class': "sc-empnci klToKr wui-text"})
    domaine = col[0].find_next("span").string
    line = [company,job,lieu,domaine,debut,link]
    return line
get_info('https://www.welcometothejungle.com/fr/companies/hunteed/jobs/community-success-manager-community-builder_paris?q=6d2ebe57905edc93351a1b0e81bdf060&o=802986be-3cbd-4743-8bd2-b6830b8a75a2')

['Hunteed',
 'Customer Success Manager B2C H/F',
 'Paris',
 'SaaS / Cloud Services, Recrutement, Digital',
 'il y a 15 heures',
 'https://www.welcometothejungle.com/fr/companies/hunteed/jobs/community-success-manager-community-builder_paris?q=6d2ebe57905edc93351a1b0e81bdf060&o=802986be-3cbd-4743-8bd2-b6830b8a75a2']

##### Test sur une autre annonce

In [13]:
url2 = 'https://www.welcometothejungle.com/fr/companies/favikon/jobs/charge-de-marketing-communication-h-f_paris?q=09514f03310bf102a5f7e96b624cd02e&o=b4a4fa03-9df7-48a5-b038-d98148553c03'

In [14]:
get_info(url2)

['Favikon',
 'Assistant(e) de Marketing & Communication H/F - Stage',
 'Paris',
 'AdTech  / MarTech',
 'hier',
 'https://www.welcometothejungle.com/fr/companies/favikon/jobs/charge-de-marketing-communication-h-f_paris?q=09514f03310bf102a5f7e96b624cd02e&o=b4a4fa03-9df7-48a5-b038-d98148553c03']

### Fonction qui recupère toutes les infos des toutes les annonces

In [15]:
jobs_list = []
for i in range(1,35):
    all_jobs = "https://www.welcometothejungle.com/fr/jobs?query=communication&refinementList%5Boffices.country_code%5D%5B%5D=FR&page="+str(i)
    page = requests.get(all_jobs)
    soup = bs(page.text,'html.parser')
    divs = soup.find_all('div',class_="sc-bXCLTC sc-emIrwa bsJILN sc-6i2fyx-4 bpkkNp")

In [16]:
for div in divs : 
        a = div.find("a")
        if ("/fr/companies" in a["href"]) and (a["href"] not in jobs_list):
            jobs_list.append(a["href"])
            
            
jobs_list = list(set(jobs_list)) # évite les doublons
#len(jobs_list)

print(len(jobs_list))


0


#### Remarque : il y a un soucis sur cette ligne de code :  divs = soup.find_all('div',class_="sc-bXCLTC sc-emIrwa bsJILN sc-6i2fyx-4 bpkkNp") , la class n'est pas la bonne donc ça ne peut pas récupérer les infos de de toutes les offres, je ne peut pas récupérer la calss qui nous intéresse car il me semble qu'elles sont générées aléatoirement à cause du javascript.

### Création de la database

In [17]:
base = "https://www.welcometothejungle.com"
df = pd.DataFrame(columns=["Company","Job","Lieu","Domaine","Mise en ligne","Link"])

for job in jobs_list : 
    #print("Ofrre ",i)
    row = get_info(base+job)
    a = pd.DataFrame([row],columns=["Company","Job","Lieu","Domaine","Mise en ligne","Link"])
    df = pd.concat([df,a],ignore_index=True)
    
    #print("---------------------------")
df = df.drop_duplicates().reset_index(drop=True)
df

,Company,Job,Lieu,Domaine,Mise en ligne,Link


### Convertion en fichier csv

In [18]:
df.to_csv(r'./welcome_to_the¨jungle.csv', index=False, header=True)

#### Remarque: Comme ma database est vide je ne peux pas tester la suite du code

### Préparation des données 

In [19]:
categories_uniques = df['Domaine'].unique()

for category in categories_uniques:
    df[f'is_{category}'] = df['Domaine'].apply(lambda x: 1 if category in x else 0)

In [21]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(categories_uniques)
sequences = tokenizer.texts_to_sequences(categories_uniques)
max_length = max(len(seq) for seq in sequences)
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post', truncating='post')

ValueError: max() arg is an empty sequence

### Création des ensembles d'entraînement et de test

In [23]:
split_index = int(0.8 * len(categories_uniques))
train_data = padded_sequences[:split_index]
test_data = padded_sequences[split_index:]

NameError: name 'padded_sequences' is not defined

### Création du modèle

In [25]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 16
num_categories = len(categories_uniques)

model = Sequential([
    Embedding(input_dim=vocab_size, output_dim=embedding_dim, input_length=max_length),
    Bidirectional(LSTM(64)),
    Dense(64, activation='relu'),
    Dense(num_categories, activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


NameError: name 'max_length' is not defined

### Entrainement du model

In [26]:
num_epochs = 10
model.fit(train_data, epochs=num_epochs, validation_data=(test_data))

NameError: name 'model' is not defined

In [ ]:
test_loss, test_acc = model.evaluate(test_data)
print(f"Test Accuracy: {test_acc}")

### Création de l'API

In [ ]:
app = Flask(__name__)

conn = sqlite3.connect('wttj.db')
cursor = conn.cursor()

@app.route('/api/offres_filtrees', methods=['GET'])
def get_offres_filtrees():

    cursor.execute("select * from offres where Lieu='Paris'")
    result = cursor.fetchall()

### Convertir en un format JSON

In [ ]:
offres_json = []
for offre in result:
    offre_dict = {
        'Company': offre[0],
        'Job': offre[1],
        'Lieu': offre[2],
        'Domaine': offre[3],
        'Mise en ligne': offre[4],
        'Link': offre[5]
    }
    offres_json.append(offre_dict)

return jsonify({'offres_filtrees': offres_json})

### Récupération de JSON en ligne

In [ ]:
def recuperation_json(url):
    try:
        response = requests.get(url)
        
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Erreur de requête HTTP : {response.status_code}")
            return None
    except Exception as e:
        print(f"Une erreur s'est produite : {e}")
        return None
    
recuperation_json(url)